# Webスクレイピング
## 0. 概要
Webスクレイピングとは ** データ取得 ** をする際に利用する技術  
プログラムを使ってWebからコンテンツをダウンロードすること  
例： GoogleはスクレイピングでWebページの索引を作成  

### データ取得の開発フロー
1. どのようなデータが必要かを考案  
2. そのデータを取得できるAPIがあるか調査
3. APIがなかった場合スクレイピングを実装  

#### なぜスクレイピングの優先度が低いのか
* APIの方が実装コストが低い
* Webスクレイピングは実装コストがAPIと比べて高く，法律的な制限も存在する  

### スクレイピングの工程
1. Webページにアクセス
2. ログインが必要なサイトの場合: ログイン機能の実装
3. 取得したい情報の抽出
4. 定期的に取得できるように実装

### 利用モジュール
1. webbrowser, Requests
2. Selenium
3. Beautiful Soup
4. cron  

## 1. webbrowser, Requests
### (1)webbrowser
URLを指定すると新たにブラウザを起動してWebページを開くことができる

In [2]:
import webbrowser
webbrowser.open('http://inventwithpython.com/')

True

### 利用例
* 指定した住所のGoogleMapを開きたい  
検索して開くのは面倒くさい  
* 住所をスクリプトに実行するだけでGoogleMapを開くことが可能  

In [3]:
webbrowser.open("https://www.google.com/maps/place/870+Valencia+St+San+Francisco+CA/")

webbrowser.open("https://www.google.com/maps/search/%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%B8%8B%E8%B0%B7%E5%8C%BA%E3%80%80%E5%85%AB%E5%8D%83%E4%BB%A3%E3%83%93%E3%83%AB/@35.6671978,139.6575807,13z/data=!3m1!4b1")

True

### 演習
「退屈なことはPythonにやらせよう」 P.260~263を参考にして  
住所をクリップボードにコピーした状態で実行すると指定した住所のGoogleMapページが開かれるスクリプトを作成しなさい  

### 課題
以下の応用例を自分なりに実装してみなさい　 

### 応用例
* ページ上のすべてのリンクをブラウザの別々のタブで開く
* 自分の地域の天気予報を取得するURLをブラウザで開く
* 定常的にチェックする複数のSNSサイトを開く  

### (2)requests  
手軽にHTTPリクエストを送信することができるライブラリ  
ブラウザを介さずにプログラムでWebページの情報を取得することができる  
### 利用例  
* プログラムからファイルをダウンロード

In [3]:
import requests

#ダウンロードするファイルのURLを指定
url = "http://uta.pw/shodou/img/28/214.png"
savename = "test.png"

res = requests.get(url)

with open(savename, mode="wb") as f:
    f.write(res.content)
    print("保存しました")

保存しました


* Webからデータを取得してみる  

クジラ Web APIを利用して様々な情報を取得してみる  
クライアントのIPを確認するAPIを利用

In [5]:
import requests
url = "http://api.aoikujira.com/ip/ini"
res = requests.get(url)
text = res.content.decode("utf-8")
print(text)

[ip]
API_URI=http://api.aoikujira.com/ip/get.php
REMOTE_ADDR=118.109.74.141
REMOTE_HOST=FL1-118-109-74-141.tky.mesh.ad.jp
REMOTE_PORT=55828
HTTP_HOST=api.aoikujira.com
HTTP_USER_AGENT=python-requests/2.18.4
HTTP_ACCEPT_LANGUAGE=
HTTP_ACCEPT_CHARSET=
SERVER_PORT=80
FORMAT=ini




## 2. Beautiful Soupを利用したスクレイピング  
ページから一部のデータを取得  
サイト：https://www.nikkei.com/markets/kabu/   
このサイトから日経平均株価、始値、高値、安値の値を取得する  
タグを取得してパースしていく

### 1. URLからHTTPレスポンスを取得

In [9]:
import urllib.request
from bs4 import BeautifulSoup
#指定するURL
url = "https://www.nikkei.com/markets/kabu/"

html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, "html.parser")
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html dir="ltr" id="R1" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="ja" http-equiv="Content-Language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/css; charset=UTF-8" http-equiv="Content-Style-Type"/>
<meta content="text/javascript; charset=UTF-8" http-equiv="Content-Script-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="nodescription, noimage" name="mixi-check-robots"/>
<meta content="https://assets.nikkei.jp/release/v2.2.9/parts/ds/images/common/icon_ogpnikkei.png" property="og:image">
<meta content="197388106979545" property="fb:app_id"/>
<meta content="　日本経済新聞電子版の総合投資・金融情報コーナー。株式・為替から債券、商品、新興国市場など国内外の最新マーケット情報はもちろん、第一線の記者による分析・解説記事を豊富に提供します。企業の最新ニュースや詳細な株価・財務データを提供するオンライン版の「日経会社情報」、チャートや銘柄管理ツールなどの便利な機能も充実しています。" name="description"/>
<meta content

### 2. ソースから取得したい箇所を抽出していく

#### findメソッドでタグとクラスの種類を指定  
該当した最初の部分を取得することができる  

In [3]:
market = soup.find('div', class_='mk-top_stock_average cmn-clearfix')
market

<div class="mk-top_stock_average cmn-clearfix">
<div class="m-headline">
<h2 class="m-headline_text">日経平均株価<span class="cmn-notes">（12日、大引、円）</span></h2>
</div>
<div class="cmn-clearfix">
<div class="mkc-chart">
<a href="/markets/chart/#!/0101" style="text-decoration:none;" target="_blank"><img class="lazyload" data-original="https://chart-renderer.n8s.jp/m/marketdata/indexes/line/tick/?code=N101/T&amp;chart_style=pc_half&amp;term=1&amp;Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jaGFydC1yZW5kZXJlci5uOHMuanAvbS9tYXJrZXRkYXRhKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTUxNTg4OTk4MH19fV19&amp;Signature=L10lds5j1nd2cxmCKLLhQjRkcf2~gUS-CN7z3zsiAMt42TKcMEPp0CsjZejEY4TGmoii8xglzVnVlwJW2kz-cwVowM7XRlgZluGstYi8nBzav73RLRs6i4SBPLeDW5fma6YzPHcVOG3Kj4SSNq8GiwcjGpS4xECd8W4YvwU1Oa9oIzhHa-z3HMByx4yM~D031Vd-y2MehKC88mEVG0EVBhsfzpNPF73FatTdRTlyRYNVJGh-262d3ka2APAFy5eVdDrR8C6FwIy7YeyS5Sjz6j8EdujiGMTQt-9yu9YccbObcQbobG6OWXWxsVHUGI93mMBG3mkEmPnRH6JBGlbRTw__&amp;Key-Pair-Id=

#### find_allメソッドで該当する全ての部分をリストにして取得  

In [4]:
prices = market.find_all('dd')
print(prices)
print(prices[0].text)

[<dd>23,719.66</dd>, <dd>23,743.05</dd>, <dd>23,588.07</dd>]
23,719.66


## 演習 

### その1
以下のサイトの国内株式の種類とその値をリストにしましょう  
https://www.nikkei.com/markets/worldidx/  

### その2
テーブルを辞書型にして保存してみよう。  
dictionary {"日経平均株価": 23710.22, ....}  

## マスター課題
自分で集めたいデータが入っているサイトをスクレイピングしてみましょう。  
また、可能であればそのデータをグラフにまとめたりしてみましょう。  

例：　　
* お酒の銘柄と詳細な内容をスクレイピングで取ってきて、お酒ごとの関係を分析
* サッカーのワールドカップの対戦結果をスクレイピングで集め、国ごとに得点率、失点率を分析
* RETRIP（旅キュレーションサービス）から話題になっている記事のスポットをスクレイピングして、そのスポットをマッピング


In [34]:
import urllib.request
from bs4 import BeautifulSoup
#指定するURL
url = "https://www.nikkei.com/markets/worldidx/"

html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, "html.parser")

market = soup.select('table.cmn-table_style1 tr')

dict1 = {}
for text in market:
    dict1[text.find("th")] = text.select("td")
    
for key, value in dict1.items():
    print(key)
    print(value)
    print("---------------------------------")

[<tr>
<th><a href="/markets/worldidx/chart/nk225/">日経平均（円）</a></th>
<td>21,658.15</td>
<td><span class="cmn-minus">－98.40(－0.45%)</span></td>
<td>26日 15:15 大引</td>
</tr>, <tr>
<th><a href="/markets/worldidx/chart/jpxnk400/">JPX日経400</a></th>
<td>13,972.14</td>
<td><span class="cmn-minus">－53.70(－0.38%)</span></td>
<td>26日 15:00 大引</td>
</tr>, <tr>
<th><a href="/markets/kabu/nidxprice/?StockIndex=N300">日経300</a></th>
<td>319.71</td>
<td><span class="cmn-minus">－0.95(－0.30%)</span></td>
<td>26日 15:15 大引</td>
</tr>, <tr>
<th><a href="/markets/kabu/nidxprice/?StockIndex=N500">日経500平均（円）</a></th>
<td>2,035.37</td>
<td><span class="cmn-minus">－8.45(－0.41%)</span></td>
<td>26日 15:15 大引</td>
</tr>, <tr>
<th>日経JAPAN1000</th>
<td>1,868.37</td>
<td><span class="cmn-minus">－7.46(－0.40%)</span></td>
<td>26日 15:45 大引</td>
</tr>, <tr>
<th>日経ジャスダック平均（円）</th>
<td>3,471.09</td>
<td><span class="cmn-minus">－1.06(－0.03%)</span></td>
<td>26日 15:00 大引</td>
</tr>, <tr>
<th>日経平均先物（円）<br>大取,19/09月<span class="